In [ ]:
try:
    import json
    import os
    
    import pandas as  pd
    import spacy
    
    import seaborn as sns
    import string

    from tqdm import tqdm
    from textblob import TextBlob
    
    from nltk.corpus import stopwords
    import nltk
    from nltk.stem import WordNetLemmatizer
    from nltk import word_tokenize
    from nltk.corpus import stopwords
    from nltk.stem import PorterStemmer
    import re
    
    import matplotlib.pyplot as plt
    
    from sklearn.model_selection import train_test_split
    from sklearn.preprocessing import LabelEncoder
    from sklearn.feature_extraction.text import CountVectorizer
    from sklearn.feature_extraction.text import TfidfTransformer
    from sklearn.naive_bayes import MultinomialNB
    from sklearn.pipeline import Pipeline
    from sklearn.feature_extraction.text import TfidfVectorizer
    from sklearn.pipeline import Pipeline
    from sklearn.metrics import classification_report, confusion_matrix
    from sklearn.model_selection import train_test_split
    from sklearn.feature_selection import SelectKBest, chi2
    from sklearn.ensemble import RandomForestClassifier
    
    
    from sklearn.preprocessing import FunctionTransformer
    from sklearn.base import BaseEstimator, TransformerMixin
    from sklearn.pipeline import FeatureUnion
    from sklearn.feature_extraction import DictVectorizer
    
    import swifter

    import numpy as np
    
    tqdm.pandas()
except Exception as e:
    print("Error : {} ".format(e))

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

In [ ]:
csv_path = input("Enter the csv file path: ")

df = pd.read_csv("", low_memory=False)

df = df.astype(str)

df = df.sample(frac=1).reset_index(drop=True)

In [ ]:
df = df[["text", "category"]]

print("Number of duplicates:", df.duplicated().sum())

df.drop_duplicates(keep="first", inplace=True)

print("Number of duplicates:", df.duplicated().sum())

In [ ]:
print("The number of null entries are:", df.isnull().sum())

df = df.dropna()

print("The number of null entries are:", df.isnull().sum())

In [ ]:
df['category'].value_counts().plot( kind='bar', figsize=(15,10))

In [ ]:
print("The shape of the dataset is", df.shape)

In [ ]:
df.describe()

In [ ]:
df['category'].unique()

In [ ]:
stop_words_ = set(stopwords.words('english'))
wn = WordNetLemmatizer()

# List of your own stopwords
my_sw = ['make', 'amp',  'news','new' ,'time', 'u','s', 'photos',  'get', 'say']

def black_txt(token):
    return  token not in stop_words_ and token not in list(string.punctuation)  and len(token)>2 and token not in my_sw
  
def clean_txt(text):
    clean_text = []
    clean_text2 = []
    text = re.sub("'", "",text)
    text=re.sub("(\\d|\\W)+"," ",text)    
    clean_text = [ wn.lemmatize(word, pos="v") for word in word_tokenize(text.lower()) if black_txt(word)]
    clean_text2 = [word for word in clean_text if black_txt(word)]
    return " ".join(clean_text2)

In [ ]:
def subj_txt(text):
    return  TextBlob(text).sentiment[1]

def polarity_txt(text):
    return TextBlob(text).sentiment[0]

def len_text(text):
    if len(text.split())>0:
         return len(set(clean_txt(text).split()))/ len(text.split())
    else:
         return 0

In [ ]:
df['text'] = df['text'].swifter.apply(clean_txt)
df['polarity'] = df['text'].swifter.apply(polarity_txt)
df['subjectivity'] = df['text'].swifter.apply(subj_txt)
df['len'] = df['text'].swifter.apply(lambda x: len(x))

In [ ]:
X = df[['text', 'polarity', 'subjectivity','len']]
y = df['category']

encoder = LabelEncoder()
y = encoder.fit_transform(y)

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.1, stratify=y)
v = dict(zip(list(y), df['category'].to_list()))

In [ ]:
text_clf = Pipeline([
    ('vect', CountVectorizer(analyzer="word", stop_words="english")),
    ('tfidf', TfidfTransformer(use_idf=True)),
    ('chi',  SelectKBest(chi2, k=1200)),
    ('clf', RandomForestClassifier())
])

In [ ]:
text_clf.fit(x_train['text'].to_list(), list(y_train))

In [ ]:
X_TEST = x_test['text'].to_list()
Y_TEST = list(y_test)

In [ ]:
predicted = text_clf.predict(X_TEST)

In [ ]:
c = 0

for dec, category in zip(X_TEST, predicted):
    
    if c == 2:break
    
    print("-"*55)
    print(dec)
    print(v[category])
    print("-"*55)

    c = c + 1 

In [ ]:
print("Accuracy:", np.mean(predicted == Y_TEST))

In [ ]:
title_plus_des = ["Text for prediction"]
predicted = text_clf.predict(title_plus_des)
v, v[predicted[0]]

In [ ]:
import pickle
with open('models/model_1.pkl','wb') as f:
    pickle.dump(text_clf,f)

In [ ]:
import pickle
with open('models/model_1.pkl', 'rb') as f:
    clf2 = pickle.load(f)

In [ ]:
text = ["Text for prediction"]
predicted = clf2.predict_proba(text)
print(predicted[0])
v[clf2.predict(text)[0]]